# Arcjet Redact

The [Arcjet](https://arcjet.com) redact integration allows you to redact sensitive user information from your prompts before sending it to an LLM.

Arcjet Redact runs entirely on your own machine and never sends data anywhere else, ensuring best in class privacy and performance.

The Arcjet Redact object is not an LLM itself, instead it wraps an LLM. It redacts the text that is inputted to it and then unredacts the output of the wrapped LLM before returning it. 



## Overview
### Integration details

| Class | Package | Local | Serializable | PY Support | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| Arcjet | @langchain/community | ❌ | ✅ | ❌ | ![NPM - Downloads](https://img.shields.io/npm/dm/@langchain/community?style=flat-square&label=%20&) | ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

### Installation

Install the Arcjet Redaction Library:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @arcjet/redact
</Npm2Yarn>

```

And install LangChain Community:


```{=mdx}
<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core
</Npm2Yarn>

And now you're ready to start protecting your LLM calls with Arcjet Redaction!

```

## Usage

In [1]:
import {
  ArcjetRedact,
  ArcjetSensitiveInfoType,
} from "@langchain/community/llms/arcjet";
import { OpenAI } from "@langchain/openai";

// Create an instance of another LLM for Arcjet to wrap
const openai = new OpenAI({
  model: "gpt-3.5-turbo-instruct",
  openAIApiKey: process.env.OPENAI_API_KEY,
});

const arcjetRedactOptions = {
  // Specify a LLM that Arcjet Redact will call once it has redacted the input.
  llm: openai,

  // Specify the list of entities that should be redacted.
  // If this isn't specified then all entities will be redacted.
  entities: ["email", "phone-number", "ip-address", "credit-card"] as ArcjetSensitiveInfoType[],

  // You can provide a custom detect function to detect entities that we don't support yet.
  // It takes a list of tokens and you return a list of identified types or undefined.
  // The undefined types that you return should be added to the entities list if used.
  detect: (tokens: string[]) => {
    return tokens.map((t) => t === "some-sensitive-info" ? "custom-entity" : undefined)
  },

  // The number of tokens to provide to the custom detect function. This defaults to 1.
  // It can be used to provide additional context when detecting custom entity types.
  contextWindowSize: 1,

  // This allows you to provide custom replacements when redacting. Please ensure
  // that the replacements are unique so that unredaction works as expected.
  replace: (identifiedType: string) => {
    return identifiedType === "email" ? "redacted@example.com" : undefined;
  },

};

const arcjetRedact = new ArcjetRedact(arcjetRedactOptions);
const response = await arcjetRedact.invoke(
  "My email address is test@example.com, here is some-sensitive-info"
);